In [4]:
import numpy as np
import torch

import os

from data.adapter import load_gwhd_df, GwhdToYoloAdapter

print("Checking environment variables...")
assert 'PROJ_PATH' in os.environ
assert 'YOLOV7_ROOT_DIR' in os.environ
assert 'DATASET_MD5' in os.environ
assert 'DATASET_ROOT_DIR' in os.environ
assert 'DATA_BUCKET' in os.environ
print("Environment variables exist.")

DATASET_ROOT_DIR = os.environ['DATASET_ROOT_DIR']

df = load_gwhd_df(os.path.join(DATASET_ROOT_DIR, 'competition_train.csv'))
df

Checking environment variables...
Environment variables exist.


In [3]:
from matplotlib import pyplot as plt
from matplotlib.patches import Rectangle

%matplotlib inline
#bboxes = t._load_target(0, 'bboxes')
#img = t._load_image(0)

img, bboxes, labels = t[0]

# Check if image and targets are loaded correctly
fig, ax = plt.subplots(1)
ax.imshow(img)
for bbox in bboxes:
    height = bbox[3] - bbox[1]
    width = bbox[2] - bbox[0]
    rect = Rectangle((bbox[0], bbox[1]), width, height, linewidth=1, edgecolor='r', facecolor='none')
    ax.add_patch(rect)

NameError: name 't' is not defined

In [ ]:
from data.dataset import WheatHeadsDataModule

m = WheatHeadsDataModule('./data/gwhd_2021', 8, None, None)